<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.875 · Deep Learning · PEC1</p>
<p style="margin: 0; text-align:right;">2018-2 · Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# PEC 1: Redes neuronales completamente conectadas

En esta práctica implementaremos una red neuronal completamente conectada de dos formas diferentes: 

<ol start="1">
  <li>Partiendo de cero utilizando únicamente la librería numpy</li>
  <li>Utilizando la librería Keras y TensorFlow</li>
</ol>

Posteriormente utilizaremos las dos implementaciones para entrenar dos redes neuronales iguales en un conjunto de datos y compararemos el rendimiento.

**Importante: Cada uno de los ejercicios puede suponer varios minutos de ejecución, por lo que la entrega debe hacerse en formato notebook y en formato html donde se vea el código y los resultados y comentarios de cada ejercicio. Para exportar el notebook a html puede hacerse desde el menú File $\to$ Download as $\to$ HTML.**

## 0. Carga de datos

El siguiente código carga los paquetes necesarios para la práctica y además lee los datos que utilizaremos para entrenar la red neuronal.

In [1]:
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

with open("data.pickle", "rb") as f:
    data = pickle.load(f)

features = data["features"]
labels = data["labels"]

train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size=0.2)

## 1. Redes neuronales utilizando numpy 

A continuación implementaremos todas las funciones necesarias para entrenar una red neuronal completamente conectada utilizando únicamente la librería numpy. El objetivo es poder entrenar una red neuronal con cualquier número de capas en la cual la última capa tendrá una única neurona con función de activación sigmoid y las demás capas cualquier número de neuronas con función de activación relu.

La siguiente figura muestra un diagrama de como implementaremos el proceso de entrenamiento de la red neuronal:

<img src="diag.png" alt="Diagrama del entrenamiento de la red neuronal" style="height: 550px;"/>

El desarrollo está estructurado en funciones básicas que se componen según el siguiente esquema:

- L_layer_model
  - initialize_parameters
  - L_model_forward
    - linear_activation_forward
      - linear_forward
      - sigmoid
      - relu
  - compute_cost
  - L_model_backward
    - linear_activation_backward
      - linear_backward
      - sigmoid_backward
      - relu_backward
  - update_parameters
- accuracy

**Notación**:
- Denotamos $L$ el número de capas de la red neuronal.
- La matriz de pesos que conecta una capa con la siguiente la denotamos con la letra $W$, mientras que el vector de bias lo denotamos con la letra $b$.
- Superíndice $[l]$ denota una cantidad asociada con la capa número $l$. 
    - Ejemplo: $a^{[L]}$ denota la salida de la capa número $L$.
    - Ejemplo: Las variables $W^{[L]}$ y $b^{[L]}$ denotan la matriz de pesos y el vector de bias que conectan la capa $L-1$ con la capa $L$ respectivamente.
- Superíndice $(i)$ denota una cantidad asociada con el ejemplo $i$-ésimo. 
    - Ejemplo: $x^{(i)}$ es el ejemplo del conjunto de entrenamiento número $i$.

### 1.1 Inicialización de parámetros

El primer paso para entrenar una red neuronal consiste en inicializar de forma aleatoria los parámetros del modelo.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Inicializar las matrices de parámetros y los vectores de bias. Las matrices de pesos se deben inicializar utilizando la distribución normal y los vectores de bias se deben inicializar con ceros. Para las matrices de pesos podéis utilizar 0.1*np.random.randn(shape) indicando el tamaño correcto.
</div>

In [2]:
def initialize_parameters(layer_dims):
    """
    Argumentos:
    layer_dims -- lista que contiene las dimensiones de cada capa de la red
    
    Devuelve:
    parameters -- diccionario que contiene los parametros "W1", "b1", ..., "WL", "bL":
                    Wl -- matriz de pesos de tamaño (layer_dims[l], layer_dims[l-1])
                    bl -- vector de bias de tamaño (layer_dims[l], 1)
    """
    
    parameters = {}
    L = len(layer_dims)

    for l in range(1, L):
        parameters['W' + str(l)] = 0.1*np.random.randn(layer_dims[l], layer_dims[l-1])
        parameters['b' + str(l)] = np.zeros([layer_dims[l], 1])
    
    return parameters

In [3]:
initialize_parameters([100, 50, 20, 10, 1])['W4'].shape

(1, 10)

### 1.2 Propagación hacia delante

En una capa concreta de una red neuronal, las entradas de las neuronas se combinan de forma lineal antes de pasar por la función de activación según la siguiente fórmula:

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}$$

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Calcular la combinación lineal de las entradas a una capa de la red neuronal.
</div>

In [4]:
def linear_forward(A, W, b):
    """
    Implementa la parte lineal de la propagación hacia delante de una capa

    Argumentos:
    A -- salida de la capa anterior (o datos de entrada): (número de neuronas de la capa anterior, número de ejemplos)
    W -- matriz de pesos: (número de neuronas de la capa actual, número de neuronas de la capa anterior)
    b -- vector de bias: (número de neuronas de la capa actual, 1)

    Devuelve:
    Z -- la entrada a la función de activación
    cache -- una tripleta que contiene "A", "W" y "b", utilizada después para la propagación hacia atrás
    """
    
    Z = np.dot(W, A) + b
    cache = (A, W, b)
    
    return Z, cache

In [5]:
A = np.array([[1],[2]])
W = np.array([[0.1, -0.1], [0.1, -0.1], [0.1, -0.1], [0.1, -0.1], [0.1, -0.1]])
b = np.array([[0], [0], [0], [0], [0]])

Z = linear_forward(A, W, b)[0]

Una vez se ha calculado la combinación lineal de las entradas de una capa se debe aplicar una función de activación no lineal antes de enviar las salidas a la siguiente capa. Si denotamos $g$ la función de activación (en nuestro caso relu o sigmoid), tenemos la siguiente fórmula:

$$A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} + b^{[l]})$$

A continuación definimos las funciones de activación que utilizaremos en la red neuronal.

In [6]:
def sigmoid(Z):
    A = 1 / (1 + np.exp(-Z))
    cache = Z
    return A, cache

def relu(Z):
    A = np.maximum(0, Z)
    cache = Z
    return A, cache

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Calcular la combinación lineal de las entradas utilizando la función implementada anteriormente y aplicar la función de activación no lineal que corresponda.
</div>

In [7]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    Implementa la propagación hacia delante de una capa incluyendo la función de activación

    Argumentos:
    A_prev -- salida de la capa anterior (o datos de entrada): 
                (número de neuronas de la capa anterior, número de ejemplos)
    W -- matriz de pesos: (número de neuronas de la capa actual, número de neuronas de la capa anterior)
    b -- vector de bias: (número de neuronas de la capa actual, 1)
    activation -- el nombre de la función de activación a utilizar en la capa: "sigmoid" o "relu"

    Devuelve:
    A -- la salida de la capa después de aplicar la función de activación
    cache -- una dupla que contiene "linear_cache" y "activation_cache", utilizada después para la propagación hacia atrás
    """
    
    Z, linear_cache = linear_forward(A_prev, W, b)
    
    if activation == "sigmoid":    
        A, activation_cache = sigmoid(Z)
    elif activation == "relu":
        A, activation_cache = relu(Z)
        
    cache = (linear_cache, activation_cache)

    return A, cache

Dados los datos de entrada, la salida de la red neuronal se calcula aplicando diferentes capas una detrás de otra. Si denotamos la última capa como $L$, la salida de la red neuronal se corresponde con la salida de la última capa $A^{[L]}$.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Calcular la salida de la red neuronal aplicando $L-1$ capas con función de activación relu y una última capa con función de activación sigmoid.
</div>

In [8]:
def L_model_forward(X, parameters):
    """
    Implementa la propagación hacia delante de la red neuronal completa
    
    Argumentos:
    X -- datos: matriz de tamaño (número de variables, número de ejemplos)
    parameters -- salida de la función initialize_parameters()
    
    Devuelve:
    AL -- salida de la red neuronal
    caches -- lista de caches que contiene todas las caches de la función linear_activation_forward(), las caches
                indexadas de 0 a L-2 corresponden a las caches de la función de activación relu y la cache indexada
                como L-1 corresponde a la cache de la función de activación sigmoid
    """

    caches = []
    A = X
    L = len(parameters) // 2
    
    # Implementa primero las L-1 capas con función de activación relu
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters["W" + str(l)],
                                             parameters["b" + str(l)], "relu")
        caches.append(cache)
    
    # Implementa la última capa con función de activación sigmoid
    AL, cache = linear_activation_forward(A, parameters["W" + str(L)],
                                          parameters["b" + str(L)], "sigmoid")
    caches.append(cache)
    
    return AL, caches

### 1.3 Función de coste

Una vez hemos obtenido la salida de la red neuronal podemos obtener un valor que mida el rendimiento de la red neuronal utilizando una función de coste $\mathcal{L}$. En nuestro caso utilizaremos la función de coste log-loss, que viene definida por la siguiente fórmula:

$$\mathcal{L} = -\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right))$$

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Calcular el valor de la función de coste log-loss dada la salida de la red neuronal junto con las etiquetas correctas.
</div>

In [47]:
def compute_cost(AL, Y):
    """
    Calcula la función de coste

    Argumentos:
    AL -- vector que contiene la salida de la red, corresponde a las probabilidades que predice la red neuronal
            para cada ejemplo: (1, número de ejemplos)
    Y -- vector con las etiquetas correctas para los datos de entrada a la red: (1, número de ejemplos)

    Devuelve:
    cost -- valor de la función de coste log-loss
    """
    
    m = Y.shape[1]

    cost = -1 / m *(np.dot(Y,np.log(AL.T)) + np.dot(1 - Y,np.log(1 - AL).T))
    cost = np.squeeze(cost)
    
    return cost

### 1.4 Propagación hacia atrás

Para entrenar una red neuronal es necesario calcular el gradiente de la función de coste repescto a los parámetros de la red, para lo cual utilizaremos la propagación hacia atrás. La propagación hacia atrás consiste en aplicar la regla de la cadena para calcular el gradiente de la función de coste paso a paso en cada capa.

Para aplicar la regla de la cadena en la parte lineal de la neurona, supongamos que ya hemos calculado la derivada $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$. Entonces, para calcular las derivadas $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$ podemos utilizar las siguientes fórmulas:

$$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T}$$
$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]}$$

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Calcular las derivadas de la parte lineal para una sola capa.
</div>

In [10]:
def linear_backward(dZ, cache):
    """
    Implementa la parte lineal de la propagación hacia atrás para una única capa

    Argumentos:
    dZ -- derivada de la función de coste con respecto a la salida lineal de la capa actual
    cache -- tripleta que contiene los valores (A_prev, W, b), provinientes de la función linear_forward

    Devuelve:
    dA_prev -- derivada de la función de coste con respecto a la salida de la capa anterior (l-1): 
                tiene el mismo tamaño que A_prev
    dW -- derivada de la función de coste con respecto a la matriz de pesos W de la capa actual (l):
                tiene el mismo tamaño que W
    db -- derivada de la función de coste con respecto al vector de bias b de la capa actual (l):
                tiene el mismo tamaño que b
    """
    
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = 1/m * (np.dot(dZ, A_prev.T))
    db = 1/m * (np.sum(dZ,axis = 1,keepdims = True))
    dA_prev = np.dot(W.T, dZ)

    return dA_prev, dW, db

El siguiente paso consiste en aplicar la regla de la cadena a la parte no lineal de las neuronas, es decir, a las funciones de activación. Para esto, si denotamos $g$ la función de activación, podemos utilizar la siguiente fórmula:

$$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]})$$

Donde $*$ indica el producto componente a componente.

A continuación calculamos las derivadas de las funciones de activación que utilizamos en la red neuronal.

In [64]:
def sigmoid_backward(dA, cache):
    Z = cache
    s = 1 / (1 + np.exp(-Z))
    dZ = dA * s * (1 - s)
    return dZ

def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    return dZ

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Combinar el cálculo de la derivada de las funciones de activación con la derivada de la parte lineal para obtener, a partir de la derivada de la función de coste respecto la activación de una capa, la derivada de la función de coste respecto a los parámetros de la capa y respecto a las activaciones de la capa anterior.
</div>

In [12]:
def linear_activation_backward(dA, cache, activation):
    """
    Implementa la propagación hacia atrás de una única capa incluyendo la función de activación
    
    Argumentos:
    dA -- derivada de la función de coste con respecto a la salida de la capa actual (l)
    cache -- dupla que contiene "linear_cache" y "activation_cache", provinientes de la función linear_activation_forward
    activation -- el nombre de la función de activación utilizada en la capa actual (l): "sigmoid" o "relu"
    
    Devuelve:
    dA_prev -- derivada de la función de coste con respecto a la salida de la capa anterior (l-1):
                tiene el mismo tamaño que A_prev
    dW -- derivada de la función de coste con respecto a la matriz de pesos W de la capa actual (l):
                tiene el mismo tamaño que W
    db -- derivada de la función de coste con respecto al vector de bias b de la capa actual (l):
                tiene el mismo tamaño que b
    """
    
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        
    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

Por último, es posible calcular la derivada de la función de coste respecto a cualquiera de los parámetros aplicando las funciones recién implementadas empezando por la última capa. Observemos que para inicializar la propagación hacia atrás es necesario calcular primero el valor de $\frac{\partial \mathcal{L}}{\partial A^{[L]}}$.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Aplicar la propagación hacia atrás para calcular el gradiente de la función de coste. Observad que el valor de $\frac{\partial \mathcal{L}}{\partial A^{[L]}}$ viene calculado en la variable dAL y que la última capa tiene función de activación sigmoid mientras que todas las demás tienen función de activación relu.
</div>

In [54]:
def L_model_backward(AL, Y, caches):
    """
    Implementa la propagación hacia atrás de la red neuronal completa
    
    Argumentos:
    AL -- salida de la red neuronal, proviene de la función L_model_forward
    Y -- vector con las etiquetas correctas para cada ejemplo del conjunto de datos: (1, número de ejemplos)
    caches -- lista de caches que contiene todas las caches de la función linear_activation_forward(), las caches
                indexadas de 0 a L-2 corresponden a las caches de la función de activación relu y la cache indexada
                como L-1 corresponde a la cache de la función de activación sigmoid
    
    Devuelve:
    grads -- Un diccionario con las derivadas de la función de coste respecto de cada variable:
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)

    # Inicialización de la propagación hacia atrás
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

    # Gradiente de la última capa
    current_cache = linear_activation_backward(dAL, caches[L-1], "sigmoid")
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = current_cache

    # Gradiente de las capas restantes
    for l in reversed(range(L-1)):

        current_cache = linear_activation_backward(grads["dA" + str(l+2)], caches[l], activation = "relu")
        dA_prev_temp, dW_temp, db_temp = current_cache
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

### 1.5 Actualización de parámetros

Una vez disponemos del gradiente de la función de coste podemos utilizar el método del descenso del gradiente para actualizar los parámetros de la red neuronal. Si denotamos $\alpha$ la velocidad de aprendizaje, las fórmulas para aplicar un paso del descenso del gradiente son:

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]}$$

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Actualizar los parámetros de la red neuronal aplicando un paso del descenso del gradiente.
</div>

In [59]:
def update_parameters(parameters, grads, learning_rate):
    """
    Actualiza los parámetros utilizando el descenso del gradiente
    
    Argumentos:
    parameters -- diccionario que contiene los parámetros de la red neuronal
    grads -- diccionario con las derivadas de la función de coste respecto a cada parámetro,
                corresponde a la salida de la función L_model_backward
    
    Devuelve:
    parameters -- diccionario con los parámetros actualizados:
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2

    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW"+ str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db"+ str(l+1)]

    return parameters

Con todo esto es posible entrenar la red neuronal combinando las funciones definidas anteriormente para aplicar diversas iteraciones del descenso del gradiente e ir actualizando los parámetros de la red de forma reiterada.

El siguiente código muestra cómo entrenar la red neuronal que hemos construido utilizando únicamente la librería numpy.

In [35]:
def L_layer_model(X, Y, layers_dims, learning_rate, num_iterations, print_cost):
    """
    Implementa una red neuronal de L capas donde las L-1 primeras capas tienen función de activación relu y 
    la última capa tiene función de activación sigmoid.
    
    Argumentos:
    X -- datos: matriz de tamaño (número de variables, número de ejemplos)
    Y -- vector con las etiquetas correctas para cada ejemplo del conjunto de datos: (1, número de ejemplos)
    layers_dims -- lista de longitud (número de capas + 1) que contiene el número de variables y el número 
                    de neuronas en cada capa, 
    learning_rate -- velocidad de aprendizaje para aplicar el método del descenso del gradiente
    num_iterations -- número de pasos para aplicar el descenso del gradiente
    print_cost -- si el valor es True, escribe el valor de la función de coste cada 10 iteraciones
    
    Devuelve:
    parameters -- parámetros ajustados de la red neuronal
    """
    
    # Inicialización de los parámetros
    parameters = initialize_parameters(layers_dims)
    
    for i in range(0, num_iterations):
        # Propagación hacia delante
        AL, caches = L_model_forward(X, parameters)
        
        # Cálculo de la función de coste
        cost = compute_cost(AL, Y)

        # Propagación hacia atrás
        grads = L_model_backward(AL, Y, caches)
 
        # Actualización de parámetros
        parameters = update_parameters(parameters, grads, learning_rate)
                
        # Escribe el valor de la función de coste cada 10 iteraciones
        if print_cost and i % 10 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    return parameters

## 2. Redes neuronales utilizando Keras

A continuación definiremos una red neuronal completamente conectada igual a la que hemos implementado anteriormente pero esta vez utilizando la librería Keras.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Definir una red neuronal completamente conectada a partir de una lista que contiene el número de neuronas que debe tener cada capa de la red. Las primeras capas deben tener función de activación relu y la última capa debe tener función de activación sigmoid. Todas ellas tienen que tener kernel_initializer="random_normal" y bias_initializer="zeros".
</div>

In [90]:
import tensorflow.python.keras
from tensorflow.python.keras import backend
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.optimizers import SGD

def keras_model(layers_dims, learning_rate):
    """
    Crea, utilizando Keras, una red neuronal de L capas completamente conectadas donde las L-1 primeras capas
    tienen función de activación relu y la última capa tiene función de activación sigmoid.
    
    Argumentos:
    layers_dims -- lista de longitud (número de capas + 1) que contiene el número de variables y el número 
                    de neuronas en cada capa, 
    learning_rate -- velocidad de aprendizaje para aplicar el método del descenso del gradiente
    
    Devuelve:
    modelo -- objeto de Keras que representa la red neuronal
    """
    
    L = len(layers_dims)
    
    model = Sequential()
    
    # Añadir L-1 capas con función de activación relu y una última capa con función de activación sigmoid,
    # cada capa debe tener el número de neuronas indicado en la variable layers_dims, el tamaño de la capa
    # de entrada viene dado en layers_dims[0]
    for l in range(L-1):
        model.add(Dense(units=layers_dims[l+1], activation='relu', input_dim=layers_dims[l]))
    model.add(Dense(units=1, activation='sigmoid'))
    
    model.compile(optimizer=SGD(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## 3. Entrenamiento de la red neuronal

Con todas las funciones implementadas anteriormente es posible entrenar una red neuronal completamente conectada con cualquier número de capas y cualquier número de neuronas en cada capa.

A continuación definimos la estructura de capas que tendrá la red neuronal.

In [110]:
layers_dims = [100, 20, 5, 1]

Para entrenar la red neuronal que hemos construido únicamente utilizando numpy debemos ejecutar el siguiente código:

In [111]:
parameters = L_layer_model(train_x.T, train_y.reshape(1, -1), layers_dims=layers_dims, learning_rate=0.1, 
                           num_iterations=250, print_cost=True)

Cost after iteration 0: 0.686576
Cost after iteration 10: 0.669496
Cost after iteration 20: 0.629139
Cost after iteration 30: 0.527649
Cost after iteration 40: 0.353589
Cost after iteration 50: 0.219466
Cost after iteration 60: 0.153225
Cost after iteration 70: 0.120165
Cost after iteration 80: 0.101334
Cost after iteration 90: 0.089036
Cost after iteration 100: 0.080168
Cost after iteration 110: 0.073465
Cost after iteration 120: 0.068220
Cost after iteration 130: 0.063958
Cost after iteration 140: 0.060300
Cost after iteration 150: 0.057145
Cost after iteration 160: 0.054354
Cost after iteration 170: 0.051922
Cost after iteration 180: 0.049698
Cost after iteration 190: 0.047661
Cost after iteration 200: 0.045819
Cost after iteration 210: 0.044137
Cost after iteration 220: 0.042589
Cost after iteration 230: 0.041168
Cost after iteration 240: 0.039842


Para entrenar la red neuronal que hemos construido utilizando Keras debemos ejecutar el siguiente código:

In [112]:
model = keras_model(layers_dims = layers_dims, learning_rate = 0.1)
model.fit(train_x, train_y, epochs=250, batch_size=train_x.shape[0], verbose=2)

Epoch 1/250
 - 1s - loss: 0.6885 - acc: 0.5556
Epoch 2/250
 - 0s - loss: 0.6344 - acc: 0.6075
Epoch 3/250
 - 0s - loss: 0.5941 - acc: 0.6606
Epoch 4/250
 - 0s - loss: 0.5617 - acc: 0.7006
Epoch 5/250
 - 0s - loss: 0.5346 - acc: 0.7337
Epoch 6/250
 - 0s - loss: 0.5116 - acc: 0.7588
Epoch 7/250
 - 0s - loss: 0.4920 - acc: 0.7850
Epoch 8/250
 - 0s - loss: 0.4750 - acc: 0.7975
Epoch 9/250
 - 0s - loss: 0.4602 - acc: 0.8150
Epoch 10/250
 - 0s - loss: 0.4471 - acc: 0.8319
Epoch 11/250
 - 0s - loss: 0.4354 - acc: 0.8500
Epoch 12/250
 - 0s - loss: 0.4247 - acc: 0.8612
Epoch 13/250
 - 0s - loss: 0.4149 - acc: 0.8712
Epoch 14/250
 - 0s - loss: 0.4058 - acc: 0.8825
Epoch 15/250
 - 0s - loss: 0.3974 - acc: 0.8919
Epoch 16/250
 - 0s - loss: 0.3895 - acc: 0.8988
Epoch 17/250
 - 0s - loss: 0.3821 - acc: 0.9044
Epoch 18/250
 - 0s - loss: 0.3752 - acc: 0.9119
Epoch 19/250
 - 0s - loss: 0.3686 - acc: 0.9169
Epoch 20/250
 - 0s - loss: 0.3623 - acc: 0.9219
Epoch 21/250
 - 0s - loss: 0.3564 - acc: 0.9281
E

Epoch 171/250
 - 0s - loss: 0.0987 - acc: 0.9881
Epoch 172/250
 - 0s - loss: 0.0983 - acc: 0.9881
Epoch 173/250
 - 0s - loss: 0.0978 - acc: 0.9881
Epoch 174/250
 - 0s - loss: 0.0974 - acc: 0.9881
Epoch 175/250
 - 0s - loss: 0.0969 - acc: 0.9881
Epoch 176/250
 - 0s - loss: 0.0965 - acc: 0.9881
Epoch 177/250
 - 0s - loss: 0.0960 - acc: 0.9881
Epoch 178/250
 - 0s - loss: 0.0956 - acc: 0.9881
Epoch 179/250
 - 0s - loss: 0.0952 - acc: 0.9881
Epoch 180/250
 - 0s - loss: 0.0948 - acc: 0.9881
Epoch 181/250
 - 0s - loss: 0.0943 - acc: 0.9881
Epoch 182/250
 - 0s - loss: 0.0939 - acc: 0.9881
Epoch 183/250
 - 0s - loss: 0.0935 - acc: 0.9881
Epoch 184/250
 - 0s - loss: 0.0931 - acc: 0.9887
Epoch 185/250
 - 0s - loss: 0.0927 - acc: 0.9887
Epoch 186/250
 - 0s - loss: 0.0923 - acc: 0.9887
Epoch 187/250
 - 0s - loss: 0.0919 - acc: 0.9887
Epoch 188/250
 - 0s - loss: 0.0915 - acc: 0.9887
Epoch 189/250
 - 0s - loss: 0.0911 - acc: 0.9887
Epoch 190/250
 - 0s - loss: 0.0907 - acc: 0.9887
Epoch 191/250
 - 0s 

Por último, podemos utilizar la siguiente función para calcular la precisión que obtenemos con la red neuronal construida utilizando únicamente numpy.

In [113]:
def accuracy(X, y, parameters):
    """
    Calcula la precisión de las predicciones de la red neuronal.
    
    Argumentos:
    X -- datos: matriz de tamaño (número de variables, número de ejemplos)
    parameters -- parámetros de la red neuronal entrenada
    
    Returns:
    accuracy -- valor entre 0 y 1 que representa la precisión de la red neuronal
    """
    
    m = X.shape[1]
    p = np.zeros((1,m))
    
    # Propagación hacia delante
    probas, caches = L_model_forward(X, parameters)

    # Conversión de la salida de la red a valores 0 o 1
    for i in range(0, probas.shape[1]):
        if probas[0, i] > 0.5:
            p[0, i] = 1
        else:
            p[0, i] = 0
            
    accuracy = np.sum((p == y)) / m
    
    return accuracy

A continuación se muestra la precisión obtenida tanto con la red construida con numpy como con la red construida con Keras.

In [114]:
print("Red construida con numpy")
print("Precisión {:.2f}".format(accuracy(test_x.T, test_y.reshape(1, -1), parameters)))
print("---")
print("Red construida con Keras")
print("Precisión {:.2f}".format(model.evaluate(test_x, test_y, verbose=0)[1]))

Red construida con numpy
Precisión 0.97
---
Red construida con Keras
Precisión 0.96


El siguiente código permite calcular el tiempo que tarda cada red neuronal en entrenarse.

In [115]:
%%timeit
parameters = L_layer_model(train_x.T, train_y.reshape(1, -1), layers_dims=layers_dims, 
                           learning_rate=0.1, num_iterations=250, print_cost=False)

975 ms ± 200 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [116]:
%%timeit
model = keras_model(layers_dims=layers_dims, learning_rate=0.1)
model.fit(train_x, train_y, epochs=250, batch_size=train_x.shape[0], verbose=0)

6.79 s ± 449 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Análisis:</strong> Comparar el rendimiento, tanto en tiempo de ejecución como en precisión, de las dos implementaciones de la red neuronal. Utilizar diferentes hiperparámetros en la comparación: probar con diferentes valores para las dimensiones de las capas, diferente número de capas, número de iteraciones, etc. ¿Qué factores pueden estar creando las diferencias observadas?
</div>

Basándonos en los tiempos de ejecución podemos observar como la red neuronal implementada por nosotros se ejecuta más rápido que Keras, una de las razones posibles de esto es que Keras no deja de ser un wrapper para tensorflow y tiene que cumplir con la forma de hacerlo del framework. Por otro lado, las posibilidades que alberga Keras son mucho mayores, en nuestro caso se ha dado por hecho que todas las capas excepto la última va a utilizar una función de activación ReLu.

La implementación realizada nos ayuda a entender paso a paso como se construye una red neuronal y dos de los pasos más importantes de una fully connected:

* Feedforward: propagación de los pesos y de los valores de las neuronas en las capas, la aplicación de las funciones de activación en cada capa y por último, la salida con la target predecida
* Backpropagation: como se realiza un descenso del gradiente desde la última capa hasta la primera para poder reajustar los pesos y comenzar con la optimización de la red

Los resultados de cross validation son similares para ambas lo que demuestra que hemos llegado a implementar correctamente nuestra red neuronal y optimiza según debería.

## Prueba con mas capas, neuronas y epochs

In [117]:
layers_dims = [100, 200, 50, 20, 5, 1]

In [122]:
parameters = L_layer_model(train_x.T, train_y.reshape(1, -1), layers_dims=layers_dims, learning_rate=0.05, 
                           num_iterations=500, print_cost=True)

Cost after iteration 0: 0.695833
Cost after iteration 10: 0.694710
Cost after iteration 20: 0.693958
Cost after iteration 30: 0.693364
Cost after iteration 40: 0.692828
Cost after iteration 50: 0.692281
Cost after iteration 60: 0.691679
Cost after iteration 70: 0.690980
Cost after iteration 80: 0.690120
Cost after iteration 90: 0.689021
Cost after iteration 100: 0.687551
Cost after iteration 110: 0.685525
Cost after iteration 120: 0.682624
Cost after iteration 130: 0.678299
Cost after iteration 140: 0.671505
Cost after iteration 150: 0.660153
Cost after iteration 160: 0.639951
Cost after iteration 170: 0.602494
Cost after iteration 180: 0.538479
Cost after iteration 190: 0.460864
Cost after iteration 200: 0.396583
Cost after iteration 210: 0.346930
Cost after iteration 220: 0.306022
Cost after iteration 230: 0.268449
Cost after iteration 240: 0.231245
Cost after iteration 250: 0.194543
Cost after iteration 260: 0.161735
Cost after iteration 270: 0.136425
Cost after iteration 280: 0.118

In [126]:
model = keras_model(layers_dims = layers_dims, learning_rate = 0.03)
model.fit(train_x, train_y, epochs=400, batch_size=train_x.shape[0], verbose=2)

Epoch 1/400
 - 1s - loss: 0.6740 - acc: 0.5050
Epoch 2/400
 - 0s - loss: 0.6315 - acc: 0.5175
Epoch 3/400
 - 0s - loss: 0.5972 - acc: 0.5494
Epoch 4/400
 - 0s - loss: 0.5695 - acc: 0.6044
Epoch 5/400
 - 0s - loss: 0.5480 - acc: 0.6644
Epoch 6/400
 - 0s - loss: 0.5303 - acc: 0.7069
Epoch 7/400
 - 0s - loss: 0.5148 - acc: 0.7450
Epoch 8/400
 - 0s - loss: 0.5012 - acc: 0.7650
Epoch 9/400
 - 0s - loss: 0.4891 - acc: 0.7837
Epoch 10/400
 - 0s - loss: 0.4781 - acc: 0.8025
Epoch 11/400
 - 0s - loss: 0.4683 - acc: 0.8150
Epoch 12/400
 - 0s - loss: 0.4594 - acc: 0.8331
Epoch 13/400
 - 0s - loss: 0.4512 - acc: 0.8413
Epoch 14/400
 - 0s - loss: 0.4438 - acc: 0.8550
Epoch 15/400
 - 0s - loss: 0.4370 - acc: 0.8625
Epoch 16/400
 - 0s - loss: 0.4308 - acc: 0.8719
Epoch 17/400
 - 0s - loss: 0.4251 - acc: 0.8788
Epoch 18/400
 - 0s - loss: 0.4197 - acc: 0.8875
Epoch 19/400
 - 0s - loss: 0.4148 - acc: 0.8931
Epoch 20/400
 - 0s - loss: 0.4101 - acc: 0.8988
Epoch 21/400
 - 0s - loss: 0.4057 - acc: 0.9050
E

Epoch 171/400
 - 0s - loss: 0.1990 - acc: 0.9875
Epoch 172/400
 - 0s - loss: 0.1983 - acc: 0.9875
Epoch 173/400
 - 0s - loss: 0.1977 - acc: 0.9875
Epoch 174/400
 - 0s - loss: 0.1970 - acc: 0.9875
Epoch 175/400
 - 0s - loss: 0.1964 - acc: 0.9875
Epoch 176/400
 - 0s - loss: 0.1957 - acc: 0.9875
Epoch 177/400
 - 0s - loss: 0.1951 - acc: 0.9875
Epoch 178/400
 - 0s - loss: 0.1944 - acc: 0.9875
Epoch 179/400
 - 0s - loss: 0.1938 - acc: 0.9881
Epoch 180/400
 - 0s - loss: 0.1932 - acc: 0.9881
Epoch 181/400
 - 0s - loss: 0.1926 - acc: 0.9881
Epoch 182/400
 - 0s - loss: 0.1919 - acc: 0.9881
Epoch 183/400
 - 0s - loss: 0.1913 - acc: 0.9881
Epoch 184/400
 - 0s - loss: 0.1907 - acc: 0.9881
Epoch 185/400
 - 0s - loss: 0.1901 - acc: 0.9881
Epoch 186/400
 - 0s - loss: 0.1895 - acc: 0.9881
Epoch 187/400
 - 0s - loss: 0.1889 - acc: 0.9881
Epoch 188/400
 - 0s - loss: 0.1883 - acc: 0.9881
Epoch 189/400
 - 0s - loss: 0.1877 - acc: 0.9881
Epoch 190/400
 - 0s - loss: 0.1871 - acc: 0.9881
Epoch 191/400
 - 0s 

 - 0s - loss: 0.1272 - acc: 0.9906
Epoch 339/400
 - 0s - loss: 0.1270 - acc: 0.9906
Epoch 340/400
 - 0s - loss: 0.1267 - acc: 0.9906
Epoch 341/400
 - 0s - loss: 0.1264 - acc: 0.9906
Epoch 342/400
 - 0s - loss: 0.1261 - acc: 0.9906
Epoch 343/400
 - 0s - loss: 0.1259 - acc: 0.9906
Epoch 344/400
 - 0s - loss: 0.1256 - acc: 0.9912
Epoch 345/400
 - 0s - loss: 0.1253 - acc: 0.9912
Epoch 346/400
 - 0s - loss: 0.1251 - acc: 0.9912
Epoch 347/400
 - 0s - loss: 0.1248 - acc: 0.9912
Epoch 348/400
 - 0s - loss: 0.1245 - acc: 0.9912
Epoch 349/400
 - 0s - loss: 0.1243 - acc: 0.9912
Epoch 350/400
 - 0s - loss: 0.1240 - acc: 0.9912
Epoch 351/400
 - 0s - loss: 0.1237 - acc: 0.9912
Epoch 352/400
 - 0s - loss: 0.1235 - acc: 0.9912
Epoch 353/400
 - 0s - loss: 0.1232 - acc: 0.9912
Epoch 354/400
 - 0s - loss: 0.1229 - acc: 0.9912
Epoch 355/400
 - 0s - loss: 0.1227 - acc: 0.9912
Epoch 356/400
 - 0s - loss: 0.1224 - acc: 0.9912
Epoch 357/400
 - 0s - loss: 0.1222 - acc: 0.9912
Epoch 358/400
 - 0s - loss: 0.1219

In [127]:
print("Red construida con numpy")
print("Precisión {:.2f}".format(accuracy(test_x.T, test_y.reshape(1, -1), parameters)))
print("---")
print("Red construida con Keras")
print("Precisión {:.2f}".format(model.evaluate(test_x, test_y, verbose=0)[1]))

Red construida con numpy
Precisión 0.96
---
Red construida con Keras
Precisión 0.96


Se puede observar como aumentando el número de epochs se reduce el coste en entrenamiento y aumenta el accuracy en keras (por muy poco) en comparación de valores anteriores. Por otro lado, también se ha reducido el learning rate para compensar la cantidad de los epochs y asegurar que convergiera, sin embargo en este proble parece que no se apreciarian problemas de convergencia (a menos que el learning rate fuera muy alto)

## Prueba con learning rate alto

In [130]:
parameters = L_layer_model(train_x.T, train_y.reshape(1, -1), layers_dims=layers_dims, learning_rate=1, 
                           num_iterations=250, print_cost=True)

Cost after iteration 0: 0.692636
Cost after iteration 10: 0.337021
Cost after iteration 20: 0.128266
Cost after iteration 30: 0.032389
Cost after iteration 40: 0.021307
Cost after iteration 50: 0.014673
Cost after iteration 60: 0.009729
Cost after iteration 70: 0.006679
Cost after iteration 80: 0.004324
Cost after iteration 90: 0.002659
Cost after iteration 100: 0.001803
Cost after iteration 110: 0.001319
Cost after iteration 120: 0.001018
Cost after iteration 130: 0.000813
Cost after iteration 140: 0.000669
Cost after iteration 150: 0.000564
Cost after iteration 160: 0.000485
Cost after iteration 170: 0.000423
Cost after iteration 180: 0.000375
Cost after iteration 190: 0.000335
Cost after iteration 200: 0.000302
Cost after iteration 210: 0.000274
Cost after iteration 220: 0.000251
Cost after iteration 230: 0.000231
Cost after iteration 240: 0.000213


In [131]:
model = keras_model(layers_dims = layers_dims, learning_rate = 1)
model.fit(train_x, train_y, epochs=250, batch_size=train_x.shape[0], verbose=2)

Epoch 1/250
 - 1s - loss: 0.6925 - acc: 0.5238
Epoch 2/250
 - 0s - loss: 0.6812 - acc: 0.5031
Epoch 3/250
 - 0s - loss: 0.6375 - acc: 0.7175
Epoch 4/250
 - 0s - loss: 0.5087 - acc: 0.8875
Epoch 5/250
 - 0s - loss: 0.4241 - acc: 0.8119
Epoch 6/250
 - 0s - loss: 0.7032 - acc: 0.5181
Epoch 7/250
 - 0s - loss: 0.4617 - acc: 0.8681
Epoch 8/250
 - 0s - loss: 0.3162 - acc: 0.9613
Epoch 9/250
 - 0s - loss: 0.2719 - acc: 0.9675
Epoch 10/250
 - 0s - loss: 0.2405 - acc: 0.9706
Epoch 11/250
 - 0s - loss: 0.2150 - acc: 0.9731
Epoch 12/250
 - 0s - loss: 0.1931 - acc: 0.9750
Epoch 13/250
 - 0s - loss: 0.1737 - acc: 0.9775
Epoch 14/250
 - 0s - loss: 0.1573 - acc: 0.9837
Epoch 15/250
 - 0s - loss: 0.1438 - acc: 0.9869
Epoch 16/250
 - 0s - loss: 0.1329 - acc: 0.9881
Epoch 17/250
 - 0s - loss: 0.1239 - acc: 0.9894
Epoch 18/250
 - 0s - loss: 0.1163 - acc: 0.9894
Epoch 19/250
 - 0s - loss: 0.1097 - acc: 0.9894
Epoch 20/250
 - 0s - loss: 0.1038 - acc: 0.9894
Epoch 21/250
 - 0s - loss: 0.0985 - acc: 0.9900
E

Epoch 171/250
 - 0s - loss: 0.0183 - acc: 0.9975
Epoch 172/250
 - 0s - loss: 0.0182 - acc: 0.9975
Epoch 173/250
 - 0s - loss: 0.0182 - acc: 0.9975
Epoch 174/250
 - 0s - loss: 0.0182 - acc: 0.9975
Epoch 175/250
 - 0s - loss: 0.0181 - acc: 0.9975
Epoch 176/250
 - 0s - loss: 0.0181 - acc: 0.9975
Epoch 177/250
 - 0s - loss: 0.0181 - acc: 0.9975
Epoch 178/250
 - 0s - loss: 0.0181 - acc: 0.9975
Epoch 179/250
 - 0s - loss: 0.0180 - acc: 0.9975
Epoch 180/250
 - 0s - loss: 0.0180 - acc: 0.9975
Epoch 181/250
 - 0s - loss: 0.0180 - acc: 0.9975
Epoch 182/250
 - 0s - loss: 0.0180 - acc: 0.9975
Epoch 183/250
 - 0s - loss: 0.0179 - acc: 0.9975
Epoch 184/250
 - 0s - loss: 0.0179 - acc: 0.9975
Epoch 185/250
 - 0s - loss: 0.0179 - acc: 0.9975
Epoch 186/250
 - 0s - loss: 0.0179 - acc: 0.9975
Epoch 187/250
 - 0s - loss: 0.0179 - acc: 0.9975
Epoch 188/250
 - 0s - loss: 0.0178 - acc: 0.9975
Epoch 189/250
 - 0s - loss: 0.0178 - acc: 0.9975
Epoch 190/250
 - 0s - loss: 0.0178 - acc: 0.9975
Epoch 191/250
 - 0s 

In [132]:
print("Red construida con numpy")
print("Precisión {:.2f}".format(accuracy(test_x.T, test_y.reshape(1, -1), parameters)))
print("---")
print("Red construida con Keras")
print("Precisión {:.2f}".format(model.evaluate(test_x, test_y, verbose=0)[1]))

Red construida con numpy
Precisión 0.97
---
Red construida con Keras
Precisión 0.97


Para este casos se puede observar como las redes siguen convergiendo e incluso han mejorado un poco la precisión de ambos y como se ve la cantidad de epochs es la cantidad original.

Veamos que pasaría si aumentamos drásticamente los epochs:

In [139]:
parameters = L_layer_model(train_x.T, train_y.reshape(1, -1), layers_dims=layers_dims, learning_rate=0.8, 
                           num_iterations=2000, print_cost=True)

Cost after iteration 0: 0.693159
Cost after iteration 10: 0.357389
Cost after iteration 20: 0.143485
Cost after iteration 30: 0.083377
Cost after iteration 40: 0.056070
Cost after iteration 50: 0.040931
Cost after iteration 60: 0.031653
Cost after iteration 70: 0.025340
Cost after iteration 80: 0.021198
Cost after iteration 90: 0.018200
Cost after iteration 100: 0.015942
Cost after iteration 110: 0.014182
Cost after iteration 120: 0.012772
Cost after iteration 130: 0.011620
Cost after iteration 140: 0.010656
Cost after iteration 150: 0.009837
Cost after iteration 160: 0.009138
Cost after iteration 170: 0.008531
Cost after iteration 180: 0.007998
Cost after iteration 190: 0.007529
Cost after iteration 200: 0.007112
Cost after iteration 210: 0.006738
Cost after iteration 220: 0.006401
Cost after iteration 230: 0.006098
Cost after iteration 240: 0.005821
Cost after iteration 250: 0.005568
Cost after iteration 260: 0.005336
Cost after iteration 270: 0.005123
Cost after iteration 280: 0.004

In [140]:
model = keras_model(layers_dims = layers_dims, learning_rate = 0.99)
model.fit(train_x, train_y, epochs=2000, batch_size=train_x.shape[0], verbose=2)

Epoch 1/2000
 - 1s - loss: 0.6560 - acc: 0.5275
Epoch 2/2000
 - 0s - loss: 0.4657 - acc: 0.8506
Epoch 3/2000
 - 0s - loss: 0.5955 - acc: 0.7469
Epoch 4/2000
 - 0s - loss: 0.3336 - acc: 0.9550
Epoch 5/2000
 - 0s - loss: 0.2825 - acc: 0.9638
Epoch 6/2000
 - 0s - loss: 0.2477 - acc: 0.9719
Epoch 7/2000
 - 0s - loss: 0.2203 - acc: 0.9737
Epoch 8/2000
 - 0s - loss: 0.1983 - acc: 0.9762
Epoch 9/2000
 - 0s - loss: 0.1802 - acc: 0.9794
Epoch 10/2000
 - 0s - loss: 0.1650 - acc: 0.9812
Epoch 11/2000
 - 0s - loss: 0.1522 - acc: 0.9837
Epoch 12/2000
 - 0s - loss: 0.1412 - acc: 0.9850
Epoch 13/2000
 - 0s - loss: 0.1313 - acc: 0.9856
Epoch 14/2000
 - 0s - loss: 0.1224 - acc: 0.9869
Epoch 15/2000
 - 0s - loss: 0.1150 - acc: 0.9881
Epoch 16/2000
 - 0s - loss: 0.1085 - acc: 0.9894
Epoch 17/2000
 - 0s - loss: 0.1027 - acc: 0.9894
Epoch 18/2000
 - 0s - loss: 0.0977 - acc: 0.9894
Epoch 19/2000
 - 0s - loss: 0.0932 - acc: 0.9900
Epoch 20/2000
 - 0s - loss: 0.0892 - acc: 0.9906
Epoch 21/2000
 - 0s - loss: 0

 - 0s - loss: 0.0374 - acc: 0.9931
Epoch 168/2000
 - 0s - loss: 0.0374 - acc: 0.9931
Epoch 169/2000
 - 0s - loss: 0.0373 - acc: 0.9931
Epoch 170/2000
 - 0s - loss: 0.0373 - acc: 0.9931
Epoch 171/2000
 - 0s - loss: 0.0373 - acc: 0.9931
Epoch 172/2000
 - 0s - loss: 0.0373 - acc: 0.9931
Epoch 173/2000
 - 0s - loss: 0.0372 - acc: 0.9931
Epoch 174/2000
 - 0s - loss: 0.0372 - acc: 0.9931
Epoch 175/2000
 - 0s - loss: 0.0372 - acc: 0.9931
Epoch 176/2000
 - 0s - loss: 0.0372 - acc: 0.9931
Epoch 177/2000
 - 0s - loss: 0.0372 - acc: 0.9931
Epoch 178/2000
 - 0s - loss: 0.0372 - acc: 0.9931
Epoch 179/2000
 - 0s - loss: 0.0371 - acc: 0.9931
Epoch 180/2000
 - 0s - loss: 0.0371 - acc: 0.9931
Epoch 181/2000
 - 0s - loss: 0.0371 - acc: 0.9931
Epoch 182/2000
 - 0s - loss: 0.0371 - acc: 0.9931
Epoch 183/2000
 - 0s - loss: 0.0371 - acc: 0.9931
Epoch 184/2000
 - 0s - loss: 0.0371 - acc: 0.9931
Epoch 185/2000
 - 0s - loss: 0.0371 - acc: 0.9931
Epoch 186/2000
 - 0s - loss: 0.0370 - acc: 0.9931
Epoch 187/2000


 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 332/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 333/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 334/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 335/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 336/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 337/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 338/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 339/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 340/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 341/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 342/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 343/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 344/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 345/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 346/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 347/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 348/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 349/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 350/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 351/2000


 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 496/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 497/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 498/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 499/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 500/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 501/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 502/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 503/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 504/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 505/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 506/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 507/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 508/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 509/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 510/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 511/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 512/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 513/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 514/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 515/2000


 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 660/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 661/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 662/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 663/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 664/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 665/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 666/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 667/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 668/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 669/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 670/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 671/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 672/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 673/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 674/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 675/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 676/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 677/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 678/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 679/2000


...

Epoch 1309/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1310/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1311/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1312/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1313/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1314/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1315/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1316/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1317/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1318/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1319/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1320/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1321/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1322/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1323/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1324/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1325/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1326/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1327/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1328/2000
 - 0s - loss: 0

Epoch 1470/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1471/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1472/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1473/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1474/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1475/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1476/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1477/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1478/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1479/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1480/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1481/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1482/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1483/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1484/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1485/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1486/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1487/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1488/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1489/2000
 - 0s - loss: 0

Epoch 1631/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1632/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1633/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1634/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1635/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1636/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1637/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1638/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1639/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1640/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1641/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1642/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1643/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1644/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1645/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1646/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1647/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1648/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1649/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1650/2000
 - 0s - loss: 0

Epoch 1792/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1793/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1794/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1795/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1796/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1797/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1798/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1799/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1800/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1801/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1802/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1803/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1804/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1805/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1806/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1807/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1808/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1809/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1810/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1811/2000
 - 0s - loss: 0

Epoch 1953/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1954/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1955/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1956/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1957/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1958/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1959/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1960/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1961/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1962/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1963/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1964/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1965/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1966/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1967/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1968/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1969/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1970/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1971/2000
 - 0s - loss: 0.0364 - acc: 0.9931
Epoch 1972/2000
 - 0s - loss: 0

In [141]:
print("Red construida con numpy")
print("Precisión {:.2f}".format(accuracy(test_x.T, test_y.reshape(1, -1), parameters)))
print("---")
print("Red construida con Keras")
print("Precisión {:.2f}".format(model.evaluate(test_x, test_y, verbose=0)[1]))

Red construida con numpy
Precisión 0.97
---
Red construida con Keras
Precisión 0.96


Como se puede observar aumentar la cantidad de epochs no tiene porque conllevar un aumento de la precisión y si nos fijamos en la función de coste en Keras, esta se estabiliza mucho antes de llegar a los epochs finales, para estos casos sería útil definir un *early stop* con el fin de evitar un mayor uso de recursos innecesariamente.

### References
Parte del codigo utilizado para desarrollar esta práctica proviene del curso de Coursera ["Neural networks and deep learning"](https://www.coursera.org/learn/neural-networks-deep-learning?specialization=deep-learning)